In [12]:
import os

%pwd
%cd '/home/cs/Chicken-Disease-Classification'

/home/cs/Chicken-Disease-Classification


In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path



In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory

In [15]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directory([self.config.artifacts_root])
    
    def get_prepare_callback_config(self):
        config=self.config.prepare_callbacks
        model_checkpoint_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directory([Path(model_checkpoint_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callback_config=ModelCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            
        )

        return prepare_callback_config


        

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [17]:
class PrepareCallbacK:
    def __init__(self,config: ModelCallbackConfig):
        self.config=config

    @property
    def create_tb_callback(self):
        timestamp=time.strftime("%Y-%m-%dT%H:%M-%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir,f'tb_logs_at_{timestamp}')

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def create_checkpoint_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,save_best_only=True)
    
    def get_tb_checkpoint_callback(self):
        return [self.create_tb_callback,self.create_checkpoint_callback]
    


In [18]:
try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callback_config()
    preparecallback=PrepareCallbacK(config=prepare_callback_config)
    check_list=preparecallback.get_tb_checkpoint_callback()
except Exception as e:
    raise e
    

[2024-12-11 15:39:45,261: INFO: common:Reading yaml file from:config/config.yaml]
[2024-12-11 15:39:45,267: INFO: common:Reading yaml file from:params.yaml]
[2024-12-11 15:39:45,269: INFO: common:directory created at:artifacts]
[2024-12-11 15:39:45,271: INFO: common:directory created at:artifacts/prepare_callbacks/checkpoint_dir]
[2024-12-11 15:39:45,273: INFO: common:directory created at:artifacts/prepare_callbacks/tensorboard_log_dir]
